In [231]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import MeCab
import re
import requests
from bs4 import BeautifulSoup
import geopandas as gpd

'''plot theme'''
plt.style.use('ggplot') #ggplot스타일 사용
plt.rc('axes', unicode_minus=False)
rc('font', family='AppleGothic')

# 지하철

In [2]:
sub = pd.read_csv('data/seoul_subway_data.csv')


In [3]:
sub2 = sub[['SUB_STA_NM','호선','외부코드','x_coord','y_coord']]

y = sub2['y_coord'].tolist()
x = sub2['x_coord'].tolist()

In [4]:
dong_df_sub = pd.DataFrame()
for lon, lat in zip(x,y):
    
    try:
        
        base_url = f'http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={lon},{lat}&format=xml&type=PARCEL&zipcode=false&simple=false&key=66D37555-E9A7-3294-9C22-28B005924F38'

        req = requests.get(base_url)
        soup = BeautifulSoup(req.content,'html.parser')
        body = soup.select('response > result > item')[0]

        total = body.select('text')[0].text

        deep = body.select('structure')[0]

        dong0 = deep.select('level2')[0].text
        dong1 = deep.select('level4l')[0].text
        dong2 = deep.select('level4lc')[0].text
        dong3 = deep.select('level4a')[0].text
        dong4 = deep.select('level4ac')[0].text

        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':[total]
                                ,'city':[dong0]
                                ,'dong1':[dong1]
                                ,'dong1_code':[dong2]
                                ,'dong2':[dong3]
                                ,'dong2_code':[dong4]})

        dong_df_sub = dong_df_sub.append(dong_df0)
    except:
        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':['err']
                                ,'city':['err']
                                ,'dong1':['err']
                                ,'dong1_code':['err']
                                ,'dong2':['err']
                                ,'dong2_code':['err']})
        dong_df_sub = dong_df_sub.append(dong_df0)

In [5]:
dong_df_sub

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code
0,127.044213,37.748577,경기도 의정부시 가능동 197-1,의정부시,가능동,4115011000,가능동,4115059500
0,127.118234,37.492522,서울특별시 송파구 가락동 600,송파구,가락동,1171010700,가락1동,1171063100
0,126.882343,37.481072,서울특별시 금천구 가산동 371-28,금천구,가산동,1154510100,가산동,1154551000
0,126.854456,37.561391,서울특별시 강서구 등촌동 477-1,강서구,등촌동,1150010200,,
0,126.915487,37.568491,서울특별시 서대문구 남가좌동 296-10,서대문구,남가좌동,1141012000,,
...,...,...,...,...,...,...,...,...
0,126.963672,37.559973,서울특별시 서대문구 충정로3가 334,서대문구,충정로3가,1141010200,,
0,127.006221,37.588458,서울특별시 성북구 삼선동1가 14-1,성북구,삼선동1가,1129011100,,
0,127.084689,37.620064,서울특별시 노원구 공릉동 60-1,노원구,공릉동,1135010300,,
0,126.978246,37.558514,서울특별시 중구 남창동 64-1,중구,남창동,1114011200,회현동,1114054000


# 버스

In [194]:
bus = pd.read_csv('data/seoul_bus_data.csv')
bus.rename(columns = {'X좌표':'x_coord'
                     ,'Y좌표':'y_coord'},inplace = True)
bus = bus.dropna()

bus2 = bus[['BUS_STA_NM','x_coord','y_coord']]
bus2 = bus2.drop_duplicates('BUS_STA_NM')
#bus2 = bus2.dropna()

In [196]:
bus_temp = bus.drop(columns = ['BSST_ARS_NO','x_coord','y_coord'])

In [197]:
bus_temp2 = bus_temp.groupby('BUS_STA_NM').sum().reset_index()

In [198]:
bus_temp3 = pd.merge(bus_temp2,bus2, on = 'BUS_STA_NM', how = 'left')

In [7]:
bus_coord = bus.groupby(['X좌표','Y좌표'])['MIDNIGHT_RIDE_NUM'].sum().reset_index()[['X좌표','Y좌표']]
y = bus_coord['Y좌표'].tolist()
x = bus_coord['X좌표'].tolist()

In [8]:
idx_ls = list(range(len(x)))

In [9]:
dong_df_bus = pd.DataFrame()
for lon, lat, idx in zip(x,y,idx_ls):
    
    if idx % 100 == 0:
        print('처리 : ', idx)
    try:
        
        base_url = f'http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={lon},{lat}&format=xml&type=PARCEL&zipcode=false&simple=false&key=66D37555-E9A7-3294-9C22-28B005924F38'

        req = requests.get(base_url)
        soup = BeautifulSoup(req.content,'html.parser')
        body = soup.select('response > result > item')[0]

        total = body.select('text')[0].text

        deep = body.select('structure')[0]

        dong0 = deep.select('level2')[0].text
        dong1 = deep.select('level4l')[0].text
        dong2 = deep.select('level4lc')[0].text
        dong3 = deep.select('level4a')[0].text
        dong4 = deep.select('level4ac')[0].text

        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':[total]
                                ,'city':[dong0]
                                ,'dong1':[dong1]
                                ,'dong1_code':[dong2]
                                ,'dong2':[dong3]
                                ,'dong2_code':[dong4]})

        dong_df_bus = dong_df_bus.append(dong_df0)
    except:
        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':['err']
                                ,'city':['err']
                                ,'dong1':['err']
                                ,'dong1_code':['err']
                                ,'dong2':['err']
                                ,'dong2_code':['err']})
        dong_df_bus = dong_df_bus.append(dong_df0)

처리 :  0
처리 :  100
처리 :  200
처리 :  300
처리 :  400
처리 :  500
처리 :  600
처리 :  700
처리 :  800
처리 :  900
처리 :  1000
처리 :  1100
처리 :  1200
처리 :  1300
처리 :  1400
처리 :  1500
처리 :  1600
처리 :  1700
처리 :  1800
처리 :  1900
처리 :  2000
처리 :  2100
처리 :  2200
처리 :  2300
처리 :  2400
처리 :  2500
처리 :  2600
처리 :  2700
처리 :  2800
처리 :  2900
처리 :  3000
처리 :  3100
처리 :  3200
처리 :  3300
처리 :  3400
처리 :  3500
처리 :  3600
처리 :  3700
처리 :  3800
처리 :  3900
처리 :  4000
처리 :  4100
처리 :  4200
처리 :  4300
처리 :  4400
처리 :  4500
처리 :  4600
처리 :  4700
처리 :  4800
처리 :  4900
처리 :  5000
처리 :  5100
처리 :  5200
처리 :  5300


In [11]:
dong_df_bus[dong_df_bus['city'] != 'err']

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code
0,126.925847,37.646905,err,err,err,err,err,err
0,126.926869,37.647774,err,err,err,err,err,err
0,126.929320,37.649611,err,err,err,err,err,err
0,126.932290,37.609096,err,err,err,err,err,err
0,126.935117,37.650345,err,err,err,err,err,err
0,126.936408,37.649733,err,err,err,err,err,err


In [222]:
dong_df_bus

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code
0,126.797811,37.578450,서울특별시 강서구 개화동 493-2,강서구,개화동,1150011000,방화제2동,1150064000
0,126.797978,37.578220,서울특별시 강서구 개화동 492-26,강서구,개화동,1150011000,,
0,126.798863,37.562420,서울특별시 강서구 과해동 676,강서구,과해동,1150011100,공항동,1150062000
0,126.798872,37.562433,서울특별시 강서구 과해동 676,강서구,과해동,1150011100,공항동,1150062000
0,126.799788,37.578494,서울특별시 강서구 개화동 446-1,강서구,개화동,1150011000,,
...,...,...,...,...,...,...,...,...
0,127.179400,37.568110,경기도 하남시 망월동 1035,하남시,망월동,4145010900,,
0,127.179463,37.559553,서울특별시 강동구 강일동 693,강동구,강일동,1174011000,,
0,127.179720,37.559558,서울특별시 강동구 강일동 693,강동구,강일동,1174011000,,
0,127.179900,37.561716,서울특별시 강동구 강일동 693,강동구,강일동,1174011000,,


In [202]:
bus_temp4 = pd.merge(bus_temp3,dong_df_bus, on = ['x_coord','y_coord'], how = 'left')

In [206]:
bus_temp5 = bus_temp4.drop(columns = ['total_addr','dong1','dong1_code','dong2','dong2_code'])

## 도서관 대출량 좌표 변환

In [215]:
cnt = pd.read_csv('2020_07_counting.csv')

In [216]:
cnt

,lib_name,gu,위도,경도,counts
0,양천도서관,양천구,37.533654,126.875957,45512
1,송파도서관,송파구,37.499993,127.134839,43040
2,서울시립어린이도서관,종로구,37.576300,126.968237,37082
3,서초구립반포도서관,서초구,37.502493,127.012451,37010
4,노원평생학습관,노원구,37.639396,127.067748,34034
...,...,...,...,...,...
216,신사어린이도서관,은평구,37.598276,126.912158,0
217,역마루작은도서관,은평구,37.604714,126.915099,0
218,은평구행정자료실,은평구,37.602452,126.929331,0
219,이촌2동 작은도서관,용산구,37.526243,126.954647,0


In [217]:
y = cnt['위도'].tolist()
x = cnt['경도'].tolist()

In [218]:
cnt_df = pd.DataFrame()
for lon, lat, idx in zip(x,y,idx_ls):
    
    if idx % 100 == 0:
        print('처리 : ', idx)
    try:
        
        base_url = f'http://api.vworld.kr/req/address?service=address&request=getAddress&version=2.0&crs=epsg:4326&point={lon},{lat}&format=xml&type=PARCEL&zipcode=false&simple=false&key=66D37555-E9A7-3294-9C22-28B005924F38'

        req = requests.get(base_url)
        soup = BeautifulSoup(req.content,'html.parser')
        body = soup.select('response > result > item')[0]

        total = body.select('text')[0].text

        deep = body.select('structure')[0]

        dong0 = deep.select('level2')[0].text
        dong1 = deep.select('level4l')[0].text
        dong2 = deep.select('level4lc')[0].text
        dong3 = deep.select('level4a')[0].text
        dong4 = deep.select('level4ac')[0].text

        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':[total]
                                ,'city':[dong0]
                                ,'dong1':[dong1]
                                ,'dong1_code':[dong2]
                                ,'dong2':[dong3]
                                ,'dong2_code':[dong4]})

        cnt_df = cnt_df.append(dong_df0)
    except:
        dong_df0 = pd.DataFrame({'x_coord':[lon]
                                ,'y_coord':[lat]
                                ,'total_addr':['err']
                                ,'city':['err']
                                ,'dong1':['err']
                                ,'dong1_code':['err']
                                ,'dong2':['err']
                                ,'dong2_code':['err']})
        cnt_df = cnt_df.append(dong_df0)

처리 :  0
처리 :  100
처리 :  200


In [358]:
cnt_df['lib_name'] = cnt['lib_name'].tolist()
cnt_df['counts'] = cnt['counts'].tolist()

In [360]:
cnt_df.to_csv('data/필령이형전달.csv',index = False, encoding = 'utf8')

# 동 데이터 정합성 맞추기

In [14]:
addr = pd.read_excel('data/행정_법정동 중심좌표.xlsx')
addr2 = addr[addr['시도'] == '서울특별시']
addr3 = addr2.drop_duplicates(['시군구','읍면동'])

In [133]:
dong_df_sub_notnull = dong_df_sub[dong_df_sub['city'] != 'err']
dong_df_sub_null = dong_df_sub[dong_df_sub['city'] == 'err']

dong_df_bus_notnull = bus_temp4[bus_temp4['city'] != 'err']
dong_df_bus_null = bus_temp4[bus_temp4['city'] == 'err']

In [135]:
dong_df_sub_notnull2 = dong_df_sub_notnull[dong_df_sub_notnull['dong2'] != ' ']
dong_df_sub_notnull3 = dong_df_sub_notnull[dong_df_sub_notnull['dong2'] == ' ']
#dong_df_sub_null2 = dong_df_sub_null[dong_df_sub_null['dong2'] == ' ']

dong_df_bus_notnull2 = dong_df_bus_notnull[dong_df_bus_notnull['dong2'] != ' ']
dong_df_bus_notnull3 = dong_df_bus_notnull[dong_df_bus_notnull['dong2'] == ' ']
#dong_df_bus_null2 = dong_df_bus_null[dong_df_bus_null['dong2'] == ' ']

In [138]:
sub_fin0 = pd.merge(dong_df_sub_notnull2, addr3, left_on = ['city','dong2'], right_on = ['시군구','읍면동'], how = 'left')
sub_fin1 = pd.merge(dong_df_sub_notnull3, addr3, left_on = ['city','dong1'], right_on = ['시군구','읍면동'], how = 'left')

In [139]:
bus_fin0 = pd.merge(dong_df_bus_notnull2, addr3, left_on = ['city','dong2'], right_on = ['시군구','읍면동'], how = 'left')
bus_fin1 = pd.merge(dong_df_bus_notnull3, addr3, left_on = ['city','dong1'], right_on = ['시군구','읍면동'], how = 'left')

In [140]:
sub_fin2 = pd.concat([sub_fin0,sub_fin1],axis = 0)
bus_fin2 = pd.concat([bus_fin0,bus_fin1],axis = 0)

In [141]:
sub_fin2.index = range(len(sub_fin2))
bus_fin2.index = range(len(bus_fin2))

In [40]:
temp = sub_fin2.iloc[sub_fin2.isnull()[sub_fin2.isnull()['시도'] == True].index.tolist()]
temp['total_addr'].apply(lambda x: x.split(' ')[0]).value_counts()

경기도      162
인천광역시     30
충청남도      10
강원도        6
Name: total_addr, dtype: int64

In [41]:
temp = bus_fin2.iloc[bus_fin2.isnull()[bus_fin2.isnull()['시도'] == True].index.tolist()]
temp['total_addr'].apply(lambda x: x.split(' ')[0]).value_counts()

서울특별시    7
경기도      7
Name: total_addr, dtype: int64

In [143]:
sub_fin3 = sub_fin2.iloc[sub_fin2.isnull()[sub_fin2.isnull()['시도'] == False].index.tolist()]
bus_fin3 = bus_fin2.iloc[bus_fin2.isnull()[bus_fin2.isnull()['시도'] == False].index.tolist()]

In [145]:
sub_fin4 = sub_fin3[['x_coord','y_coord','dong1','dong2','시군구','위도','경도']]

In [311]:
sub_seoul = pd.merge(sub_fin4,sub,on = ['x_coord','y_coord'], how = 'left')

In [305]:
sub_fin4

,x_coord,y_coord,dong1,dong2,시군구,위도,경도
1,127.118234,37.492522,가락동,가락1동,송파구,37.496537,127.105726
2,126.882343,37.481072,가산동,가산동,금천구,37.476681,126.883767
5,127.027926,37.497175,서초동,서초2동,서초구,37.492051,127.025083
6,127.134820,37.498079,가락동,가락2동,송파구,37.498639,127.126488
7,126.858680,37.494594,개봉동,개봉제1동,구로구,37.495650,126.887771
...,...,...,...,...,...,...,...
481,127.123572,37.538397,성내동,,강동구,37.531338,127.129174
482,126.963672,37.559973,충정로3가,,서대문구,37.562611,126.961351
483,127.006221,37.588458,삼선동1가,,성북구,37.585059,127.007942
484,127.084689,37.620064,공릉동,,노원구,37.623601,127.077388


In [208]:
#bus_fin4 = bus_fin3[['x_coord','y_coord','dong1','dong2','시군구','위도','경도']]
bus_temp5

,BUS_STA_NM,MIDNIGHT_RIDE_NUM,MIDNIGHT_ALIGHT_NUM,ONE_RIDE_NUM,ONE_ALIGHT_NUM,TWO_RIDE_NUM,TWO_ALIGHT_NUM,THREE_RIDE_NUM,THREE_ALIGHT_NUM,FOUR_RIDE_NUM,...,TWENTY_ALIGHT_NUM,TWENTY_ONE_RIDE_NUM,TWENTY_ONE_ALIGHT_NUM,TWENTY_TWO_RIDE_NUM,TWENTY_TWO_ALIGHT_NUM,TWENTY_THREE_RIDE_NUM,TWENTY_THREE_ALIGHT_NUM,x_coord,y_coord,city
0,63빌딩.가톨릭대학교여의도성모병원,22.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,668.0,1778.0,632.0,1577.0,556.0,490.0,335.0,126.937405,37.519316,영등포구
1,7단지영업소,2.0,151.0,0.0,281.0,0.0,89.0,226.0,0.0,509.0,...,100.0,131.0,100.0,85.0,87.0,3.0,89.0,127.063736,37.660278,노원구
2,AK플라자,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,50.0,52.0,32.0,35.0,16.0,40.0,9.0,126.882557,37.501827,구로구
3,AT센터.양재꽃시장,5.0,424.0,2.0,109.0,0.0,0.0,0.0,0.0,0.0,...,880.0,11.0,714.0,10.0,866.0,10.0,674.0,127.039655,37.468707,서초구
4,BC카드.남부터미널,465.0,177.0,119.0,65.0,76.0,34.0,103.0,28.0,77.0,...,599.0,2708.0,530.0,2490.0,470.0,1737.0,285.0,127.014218,37.484732,서초구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3503,흥안운수종점,1.0,406.0,0.0,54.0,0.0,0.0,0.0,0.0,229.0,...,789.0,634.0,799.0,470.0,913.0,323.0,472.0,127.080722,37.669830,노원구
3504,흥진빌라,2.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,97.0,2.0,157.0,0.0,69.0,2.0,48.0,127.052376,37.622325,노원구
3505,흰돌교회,8.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,...,173.0,160.0,203.0,79.0,174.0,63.0,47.0,126.903257,37.490543,영등포구
3506,힐스테이트송파.송파한라비발디,17.0,166.0,12.0,28.0,0.0,0.0,0.0,0.0,0.0,...,330.0,7.0,333.0,2.0,260.0,9.0,270.0,127.141992,37.479484,송파구


In [316]:
sub_col = sub_seoul.columns.tolist()

sub_ride = []
sub_alight = []
for i in sub_col:
    if 'RIDE' in i:
        sub_ride.append(i)
    elif 'ALIGHT' in i:
        sub_alight.append(i)
    else:
        pass

In [209]:
bus_col = bus_temp5.columns.tolist()

bus_ride = []
bus_alight = []
for i in bus_col:
    if 'RIDE' in i:
        bus_ride.append(i)
    elif 'ALIGHT' in i:
        bus_alight.append(i)
    else:
        pass

In [318]:
sub_ride = sub_seoul[['SUB_STA_NM','x_coord','y_coord','dong1','dong2','시군구'] + sub_ride]
sub_alight = sub_seoul[['SUB_STA_NM','x_coord','y_coord','dong1','dong2','시군구'] + sub_alight]

num_ls = sub_ride.columns.tolist()[6:]
num_ls2 = sub_alight.columns.tolist()[6:]

temp = sub_ride[['SUB_STA_NM','x_coord','y_coord','dong1','dong2','시군구']]

for i in range(len(num_ls)):
    col_nm = num_ls[i].split('_')[0] + '_NUM'
    temp[col_nm] = sub_ride[num_ls[i]] + sub_alight[num_ls2[i]]
    
temp_col = temp.columns.tolist()[6:]

rush_ls = []
non_rush_ls = []
for col in temp_col:
    col_temp = col.split('_')[0]
    
    if col_temp in ['SEVEN','EIGHT','NINE','SEVENTEEN','EIGHTEEN','NINETEEN']:
        rush_ls.append(col)
    else:
        non_rush_ls.append(col)
        
temp['rush'] = temp[rush_ls].mean(axis = 1)
temp['non_rush'] = temp[non_rush_ls].mean(axis = 1)

#sub_ride_final = temp[['SUB_STA_NM','x_coord','y_coord','rush','non_rush']]
sub_seoul_ride_final = temp[['SUB_STA_NM','x_coord','y_coord','dong1','dong2','시군구','rush','non_rush']]

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [338]:
aa_temp = pd.merge(sub_seoul_ride_final,seoul, left_on = ['dong1','시군구'], right_on = ['emd_kor_nm','gu'], how = 'left')

In [343]:
aa_temp = pd.merge(seoul,sub_seoul_ride_final, left_on = ['emd_kor_nm','gu'], right_on = ['dong1','시군구'], how = 'left').dropna()

In [348]:
subway_seoul_shp = aa_temp[['SUB_STA_NM','x_coord','y_coord','gu','rush','non_rush','emd_kor_nm','geometry']]

In [361]:
subway_seoul_shp

,SUB_STA_NM,x_coord,y_coord,gu,rush,non_rush,emd_kor_nm,geometry
9,안국,126.985443,37.576477,종로구,101421.166667,26425.933333,경운동,"POLYGON ((126.98628 37.57688, 126.98690 37.575..."
16,종각,126.982923,37.570161,종로구,187598.000000,44933.333333,종로1가,"POLYGON ((126.98205 37.57051, 126.98315 37.570..."
35,종로3가,126.991806,37.571607,종로구,194805.333333,90678.066667,묘동,"POLYGON ((126.99155 37.57420, 126.99229 37.572..."
42,마곡,126.825448,37.560183,강서구,40337.500000,10700.133333,마곡동,"POLYGON ((126.84241 37.57926, 126.83753 37.570..."
43,마곡나루,126.829497,37.567336,강서구,89691.166667,18869.066667,마곡동,"POLYGON ((126.84241 37.57926, 126.83753 37.570..."
...,...,...,...,...,...,...,...,...
555,신정,126.856191,37.524997,양천구,61263.666667,15690.266667,신정동,"POLYGON ((126.86398 37.52964, 126.86428 37.526..."
556,압구정,127.028461,37.527072,강남구,150612.500000,46759.533333,신사동,"MULTIPOLYGON (((127.02191 37.51711, 127.01953 ..."
557,압구정로데오,127.040534,37.527381,강남구,86831.833333,25375.733333,신사동,"MULTIPOLYGON (((127.02191 37.51711, 127.01953 ..."
558,응암,126.915577,37.598605,은평구,80063.333333,23790.000000,신사동,"POLYGON ((126.90211 37.60374, 126.90492 37.603..."


In [362]:
# 지하철 데이터도 이 과정 거쳐야 함
check_ls = (subway_seoul_shp['emd_kor_nm'] + '|' +subway_seoul_shp['gu']).tolist()
check_ls2 = (seoul['emd_kor_nm'] +'|'+ seoul['gu']).tolist()

non_ls = []
idx_ls = []
for i in range(len(seoul)):
    
    if check_ls2[i] in check_ls:
        pass
    else:
        non_ls.append(check_ls2[i])
        idx_ls.append(i)
        
seoul_non = seoul.iloc[idx_ls]
subway_seoul_shp2 = pd.concat([subway_seoul_shp,seoul_non[['emd_kor_nm','gu','geometry']]],axis = 0)

In [367]:
subway_seoul_shp2.to_file("seoul_sub_ride.shp",encoding = 'utf8')

In [ ]:
sub_seoul_ride_final.to_csv('sub_seoul_ride_final.csv')

In [211]:
sub_ride = bus_temp5[['BUS_STA_NM','x_coord','y_coord','city'] + bus_ride]
sub_alight = bus_temp5[['BUS_STA_NM','x_coord','y_coord','city'] + bus_alight]

num_ls = sub_ride.columns.tolist()[4:]
num_ls2 = sub_alight.columns.tolist()[4:]

temp = sub_ride[['BUS_STA_NM','x_coord','y_coord','city']]

for i in range(len(num_ls)):
    col_nm = num_ls[i].split('_')[0] + '_NUM'
    temp[col_nm] = sub_ride[num_ls[i]] + sub_alight[num_ls2[i]]
    
temp_col = temp.columns.tolist()[4:]

rush_ls = []
non_rush_ls = []
for col in temp_col:
    col_temp = col.split('_')[0]
    
    if col_temp in ['SEVEN','EIGHT','NINE','SEVENTEEN','EIGHTEEN','NINETEEN']:
        rush_ls.append(col)
    else:
        non_rush_ls.append(col)
        
temp['rush'] = temp[rush_ls].mean(axis = 1)
temp['non_rush'] = temp[non_rush_ls].mean(axis = 1)

bus_ride_final = temp[['BUS_STA_NM','x_coord','y_coord','city','rush','non_rush']]

/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/sinmin-yong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [226]:
bus_ride_final

,BUS_STA_NM,x_coord,y_coord,city,rush,non_rush
0,63빌딩.가톨릭대학교여의도성모병원,126.937405,37.519316,영등포구,5976.000000,1641.800000
1,7단지영업소,127.063736,37.660278,노원구,403.500000,232.066667
2,AK플라자,126.882557,37.501827,구로구,352.666667,85.866667
3,AT센터.양재꽃시장,127.039655,37.468707,서초구,2115.833333,552.133333
4,BC카드.남부터미널,127.014218,37.484732,서초구,3795.000000,1367.800000
...,...,...,...,...,...,...
3503,흥안운수종점,127.080722,37.669830,노원구,2005.000000,1042.133333
3504,흥진빌라,127.052376,37.622325,노원구,90.333333,35.733333
3505,흰돌교회,126.903257,37.490543,영등포구,468.833333,187.600000
3506,힐스테이트송파.송파한라비발디,127.141992,37.479484,송파구,321.333333,130.400000


In [227]:
dong_df_bus

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code
0,126.797811,37.578450,서울특별시 강서구 개화동 493-2,강서구,개화동,1150011000,방화제2동,1150064000
0,126.797978,37.578220,서울특별시 강서구 개화동 492-26,강서구,개화동,1150011000,,
0,126.798863,37.562420,서울특별시 강서구 과해동 676,강서구,과해동,1150011100,공항동,1150062000
0,126.798872,37.562433,서울특별시 강서구 과해동 676,강서구,과해동,1150011100,공항동,1150062000
0,126.799788,37.578494,서울특별시 강서구 개화동 446-1,강서구,개화동,1150011000,,
...,...,...,...,...,...,...,...,...
0,127.179400,37.568110,경기도 하남시 망월동 1035,하남시,망월동,4145010900,,
0,127.179463,37.559553,서울특별시 강동구 강일동 693,강동구,강일동,1174011000,,
0,127.179720,37.559558,서울특별시 강동구 강일동 693,강동구,강일동,1174011000,,
0,127.179900,37.561716,서울특별시 강동구 강일동 693,강동구,강일동,1174011000,,


In [229]:
bus_ride_final_temp = pd.merge(bus_ride_final,dong_df_bus, on = ['x_coord','y_coord'], how = 'left')

In [232]:
# 좌표계 데이터 불러오기
seoul = gpd.read_file("seoul_total_dong_polygon.shp")

In [254]:
seoul_bus_temp = pd.merge(seoul,bus_ride_final_temp, left_on = 'emd_kor_nm', right_on = 'dong1', how = 'left')

In [257]:
seoul_bus_temp2 = seoul_bus_temp.drop_duplicates(['BUS_STA_NM','x_coord','y_coord'])
seoul_bus_temp3 = seoul_bus_temp2[['BUS_STA_NM','x_coord','y_coord','rush','non_rush','emd_kor_nm','gu','geometry']]

In [263]:
seoul_bus_temp3 = seoul_bus_temp3.dropna()

In [ ]:
check_ls = (seoul_bus_temp3['emd_kor_nm'] + '|' +seoul_bus_temp3['gu']).tolist()
check_ls2 = (seoul['emd_kor_nm'] +'|'+ seoul['gu']).tolist()

non_ls = []
idx_ls = []
for i in range(len(seoul)):
    
    if check_ls2[i] in check_ls:
        pass
    else:
        non_ls.append(check_ls2[i])
        idx_ls.append(i)
        

In [292]:
seoul_non = seoul.iloc[idx_ls]

In [295]:
seoul_bus_temp4 = pd.concat([seoul_bus_temp3,seoul_non[['emd_kor_nm','gu','geometry']]],axis = 0)

In [297]:
seoul_bus_temp4.to_file("seoul_bus_ride.shp",encoding = 'utf8')

In [214]:
with pd.ExcelWriter('data/bus_ride_alight.xlsx') as write:
    bus_ride_final.to_excel(write, sheet_name='Sheet_name_1')

In [162]:
bus_ride_final2 = pd.merge(bus_fin4,bus_ride_final, on = ['x_coord','y_coord'], how = 'left')

bus_ride_final3 = bus_ride_final2.drop_duplicates('BUS_STA_NM')

bus_ride_final4 = bus_ride_final3.groupby(['dong1','dong2','시군구','위도','경도']).agg({'rush':'sum','non_rush':'sum'})\
.reset_index() 

dong2_ls = []
for i in range(len(bus_ride_final4)):
    if bus_ride_final4['dong2'].iloc[i] == ' ':
        dong2_ls.append(bus_ride_final4['dong1'].iloc[i])
    else:
        dong2_ls.append(bus_ride_final4['dong2'].iloc[i])
        
bus_ride_final4['dong3'] = dong2_ls

bus_ride_final5 = bus_ride_final4.groupby(['dong3','시군구','위도','경도']).agg({'rush':'sum','non_rush':'sum'})\
.reset_index()

In [180]:
sub_ride_final

,SUB_STA_NM,x_coord,y_coord,rush,non_rush
0,가능,127.044213,37.748577,27475.666667,10028.533333
1,가락시장,127.118234,37.492522,76932.333333,25834.866667
2,가산디지털단지,126.882343,37.481072,368080.166667,53603.200000
3,가양,126.854456,37.561391,92158.333333,27387.133333
4,가좌,126.915487,37.568491,23130.833333,7570.133333
...,...,...,...,...,...
482,충정로,126.963672,37.559973,79019.333333,17316.333333
483,한성대입구,127.006221,37.588458,57580.833333,20963.400000
484,화랑대,127.084689,37.620064,49349.833333,13437.666667
485,회현,126.978246,37.558514,101070.166667,41438.733333


In [179]:
bus_ride_final5

,dong3,시군구,위도,경도,rush,non_rush
0,가락1동,송파구,37.496537,127.105726,10754.333333,5072.466667
1,가락2동,송파구,37.498639,127.126488,10035.000000,4201.000000
2,가락동,송파구,37.496486,127.126006,15192.333333,6257.800000
3,가락본동,송파구,37.495585,127.121830,21857.666667,8948.066667
4,가리봉동,구로구,37.482704,126.888019,27249.500000,10315.933333
...,...,...,...,...,...,...
610,후암동,용산구,37.549901,126.980613,9744.666667,3621.133333
611,휘경동,동대문구,37.589592,127.061228,29397.833333,12404.800000
612,휘경제1동,동대문구,37.574428,127.039712,968.833333,544.266667
613,휘경제2동,동대문구,37.574428,127.039712,15008.000000,6884.933333


In [182]:
with pd.ExcelWriter('data/sub_dong_final.xlsx') as write:
    sub_ride_final.to_excel(write, sheet_name='Sheet_name_1')
    
with pd.ExcelWriter('data/bus_dong_final.xlsx') as write:
    bus_ride_final5.to_excel(write, sheet_name='Sheet_name_1')

In [186]:
dong_df_sub

,x_coord,y_coord,total_addr,city,dong1,dong1_code,dong2,dong2_code
0,127.044213,37.748577,경기도 의정부시 가능동 197-1,의정부시,가능동,4115011000,가능동,4115059500
0,127.118234,37.492522,서울특별시 송파구 가락동 600,송파구,가락동,1171010700,가락1동,1171063100
0,126.882343,37.481072,서울특별시 금천구 가산동 371-28,금천구,가산동,1154510100,가산동,1154551000
0,126.854456,37.561391,서울특별시 강서구 등촌동 477-1,강서구,등촌동,1150010200,,
0,126.915487,37.568491,서울특별시 서대문구 남가좌동 296-10,서대문구,남가좌동,1141012000,,
...,...,...,...,...,...,...,...,...
0,126.963672,37.559973,서울특별시 서대문구 충정로3가 334,서대문구,충정로3가,1141010200,,
0,127.006221,37.588458,서울특별시 성북구 삼선동1가 14-1,성북구,삼선동1가,1129011100,,
0,127.084689,37.620064,서울특별시 노원구 공릉동 60-1,노원구,공릉동,1135010300,,
0,126.978246,37.558514,서울특별시 중구 남창동 64-1,중구,남창동,1114011200,회현동,1114054000


In [188]:
sub_ride_final2 = pd.merge(sub_ride_final,dong_df_sub, on = ['x_coord','y_coord'], how = 'left')

In [191]:
sub_ride_final3 = sub_ride_final2[['SUB_STA_NM','x_coord','y_coord','city','rush','non_rush']]\
.rename(columns = {'x_coord' : '경도'
                  ,'y_coord' : '위도'
                  ,'city':'시군구'})

In [184]:
bus_ride_final5

,dong3,시군구,위도,경도,rush,non_rush
0,가락1동,송파구,37.496537,127.105726,10754.333333,5072.466667
1,가락2동,송파구,37.498639,127.126488,10035.000000,4201.000000
2,가락동,송파구,37.496486,127.126006,15192.333333,6257.800000
3,가락본동,송파구,37.495585,127.121830,21857.666667,8948.066667
4,가리봉동,구로구,37.482704,126.888019,27249.500000,10315.933333
...,...,...,...,...,...,...
610,후암동,용산구,37.549901,126.980613,9744.666667,3621.133333
611,휘경동,동대문구,37.589592,127.061228,29397.833333,12404.800000
612,휘경제1동,동대문구,37.574428,127.039712,968.833333,544.266667
613,휘경제2동,동대문구,37.574428,127.039712,15008.000000,6884.933333


In [193]:
with pd.ExcelWriter('data/sub_dong_final3.xlsx') as write:
    sub_ride_final3.to_excel(write, sheet_name='Sheet_name_1')